In [10]:
from flask import Flask, request, jsonify
from datetime import datetime
from flask_cors import CORS
from supabase import create_client
import os
from dotenv import load_dotenv
import requests

load_dotenv()

url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
weatherapi_key = os.environ.get("WEATHER_API_KEY")
supabase = create_client(url, key)

In [11]:
def get_weather_api_data(lat, lon):
    """Fetch forecast from WeatherAPI"""
    url = f"http://api.weatherapi.com/v1/forecast.json"
    params = {
        "key": weatherapi_key,
        "q": f"{lat},{lon}",
        "days": 1,
        "aqi": "no"
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"WeatherAPI request failed: {response.text}")

In [14]:
forecast_data = get_weather_api_data(1.3521, 103.849)
forecast_day = forecast_data['forecast']['forecastday'][0]
formatted_data = {
    'date': forecast_day['date'],
    'hour': forecast_day['hour']
}

In [16]:
location_id = 6
record = {
    "location_id": location_id,
    "forecast_day": datetime.now().strftime("%Y-%m-%d"),
    "hourly_forecast": formatted_data["hour"]    
}

In [18]:
result = supabase.table('location_weather').insert(record).execute()
print(f"Successfully inserted forecast for location {location_id}")

Successfully inserted forecast for location 6


In [20]:
# Assuming that MS is running
BASE_URL = "http://localhost:5002"

def test_api_endpoints():
    # Test location ID
    location_id = 6
    
    # 1. Test updating forecast
    print("Testing update_forecast endpoint...")
    
    # Get weather data from WeatherAPI
    weatherapi_key = os.environ.get("WEATHER_API_KEY")
    weather_response = requests.get(
        "http://api.weatherapi.com/v1/forecast.json",
        params={
            "key": weatherapi_key,
            "q": "1.3521,103.849",  # Bishan East coordinates
            "days": 1
        }
    )
    
    if weather_response.status_code != 200:
        print(f"Failed to get weather data: {weather_response.text}")
        return
        
    # Format data for our API
    weather_data = weather_response.json()
    forecast_day = weather_data['forecast']['forecastday'][0]
    
    update_data = {
        'date': forecast_day['date'],
        'hour': forecast_day['hour']
    }
    
    # Send update request
    update_response = requests.post(
        f"{BASE_URL}/update_forecast/{location_id}",
        json=update_data
    )
    print(f"Update Response: {update_response.json()}\n")
    
    # 2. Test get latest forecast
    print("Testing get_latest_forecast endpoint...")
    latest_response = requests.get(f"{BASE_URL}/get_weather/{location_id}")
    print(f"Latest Forecast Response: {latest_response.json()}\n")
    
    # 3. Test get forecast by date
    print("Testing get_forecast_by_date endpoint...")
    date = datetime.now().strftime('%Y-%m-%d')
    date_response = requests.get(f"{BASE_URL}/get_weather/date/{location_id}/{date}")
    print(f"Date Forecast Response: {date_response.json()}\n")
    
    # 4. Test get forecast by datetime
    print("Testing get_forecast_by_datetime endpoint...")
    dt = datetime.now().strftime('%Y-%m-%d %H:00:00')
    datetime_response = requests.get(f"{BASE_URL}/get_weather/datetime/{location_id}/{dt}")
    print(f"Datetime Forecast Response: {datetime_response.json()}")

# Make sure your Flask app is running before executing this
test_api_endpoints()

Testing update_forecast endpoint...
Update Response: {'message': 'Weather forecast updated successfully'}

Testing get_latest_forecast endpoint...
Latest Forecast Response: {'forecast_day': '2025-03-13', 'hourlyForecast': [{'chance_of_rain': 0, 'chance_of_snow': 0, 'cloud': 32, 'condition': {'code': 1003, 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'text': 'Partly Cloudy '}, 'dewpoint_c': 23.8, 'dewpoint_f': 74.9, 'feelslike_c': 30.4, 'feelslike_f': 86.6, 'gust_kph': 23.4, 'gust_mph': 14.6, 'heatindex_c': 30.4, 'heatindex_f': 86.6, 'humidity': 84, 'is_day': 0, 'precip_in': 0.0, 'precip_mm': 0.0, 'pressure_in': 29.8, 'pressure_mb': 1009.0, 'snow_cm': 0.0, 'temp_c': 26.8, 'temp_f': 80.2, 'time': '2025-03-13 00:00', 'time_epoch': 1741795200, 'uv': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'will_it_rain': 0, 'will_it_snow': 0, 'wind_degree': 39, 'wind_dir': 'NE', 'wind_kph': 16.2, 'wind_mph': 10.1, 'windchill_c': 26.8, 'windchill_f': 80.2}, {'chance_of_rain': 0, 'chance_of_snow'